In [1]:
import pandas as pd
import io
import requests
import quandl, math
import numpy as np
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import datetime
import matplotlib.pyplot as plt
from matplotlib import style



c:\users\user\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df1=pd.read_csv("AAPL.csv")
df1=df1.convert_objects(convert_numeric=True)
df1["Date"]=pd.to_datetime(df1["Date"])
df1['sma5']=df1['Adj Close'].rolling(window=5).mean()
df1['sma10']=df1['Adj Close'].rolling(window=10).mean()
df1['sma_delta']=df1['sma5']-df1['sma10']
#df1['volume_SMA5']=df1['Volume'].rolling(window=5).mean()
#df1['volume_SMA10']=df1['Volume'].rolling(window=10).mean()
#df1['volume_SMA_delta']=df1['volume_SMA5']-df1['volume_SMA10']

url="https://www.quandl.com/api/v3/datasets/NS1/AAPL_US.csv?api_key=qtu8baDXX4cHaHfrVmSb"
s=requests.get(url).content
df2=pd.read_csv(io.StringIO(s.decode('utf-8')))
df2=df2.convert_objects(convert_numeric=True)
df2["Date"]=pd.to_datetime(df2.Date)
df2=df2.sort_values('Date', ascending=True)
df2=df2.reset_index(drop=True)
#df2['sentMA5']=df2['Sentiment'].rolling(window=5).mean()
#df2['sentMA10']=df2['Sentiment'].rolling(window=10).mean()
#df2['sent_delta']=df2['sentMA5']-df2['sentMA10']
df2['daily_sent_change']=(df2['Sentiment High']-df2['Sentiment Low'])

df3=df1.merge(df2,how='left',left_on='Date',right_on='Date')
df3=df3[['Adj Close','Volume','sma_delta','Sentiment','News Volume','News Buzz','daily_sent_change']]
#df3=df3[['Adj Close','Volume']]
forecast_col="Adj Close"
df3.dropna(inplace=True)
#forecast_out = int(math.ceil(0.001 * len(df3)))
forecast_out = 10

df3['label'] = df3[forecast_col].shift(-forecast_out)

X = np.array(df3.drop(['label'], 1))
X = preprocessing.scale(X)
X_lately = X[-forecast_out:]
X = X[:-forecast_out]

df3.dropna(inplace=True)

y = np.array(df3['label'])

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.3)
            
clf=LinearRegression(n_jobs=-1)
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print("Linear Regression Accuracy Is",accuracy)

forecast_set=clf.predict(X_lately)

print ("Predicted AAPL closing price for the next",forecast_out,"trading days:",forecast_set)

   


c:\users\user\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


Linear Regression Accuracy Is 0.976899160585
Predicted AAPL closing price for the next 10 trading days: [ 171.60898103  169.57156316  172.10683972  172.07163163  172.97148138
  174.7614164   174.22544024  174.26792077  174.26829313  175.3920659 ]


c:\users\user\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [3]:
print (df3[-forecast_out:])

       Adj Close      Volume       label
9330  172.669998  35273800.0  170.570007
9331  171.699997  19409200.0  170.600006
9332  172.270004  23818400.0  171.080002
9333  172.220001  20476500.0  169.229996
9334  173.970001  40169300.0  172.259995
9335  176.419998  29421100.0  172.229996
9336  174.539993  27436400.0  173.029999
9337  174.350006  23475600.0  175.000000
9338  175.009995  20949900.0  174.350006
9339  175.009995  16349400.0  174.330002


In [4]:
print (df2.tail())

           Date  Sentiment  Sentiment High  Sentiment Low  News Volume  \
1830 2018-01-05        4.0             5.0            4.0         64.0   
1831 2018-01-06        4.0             5.0            3.0         43.0   
1832 2018-01-07        5.0             5.0            4.0          9.0   
1833 2018-01-08        4.0             5.0            4.0         74.0   
1834 2018-01-09        4.0             4.0            4.0         52.0   

      News Buzz  daily_sent_change  
1830        9.0                1.0  
1831       10.0                2.0  
1832       10.0                1.0  
1833       10.0                1.0  
1834        8.0                0.0  


In [5]:
print (df1.tail())

           Date        Open        High         Low       Close   Adj Close  \
9345 2018-01-03  172.529999  174.550003  171.960007  172.229996  172.229996   
9346 2018-01-04  172.539993  173.470001  172.080002  173.029999  173.029999   
9347 2018-01-05  173.440002  175.369995  173.050003  175.000000  175.000000   
9348 2018-01-08  174.350006  175.610001  173.929993  174.350006  174.350006   
9349 2018-01-09  174.550003  175.059998  173.410004  174.330002  174.330002   

          Volume        sma5       sma10  sma_delta  
9345  29517900.0  171.079999  172.487999  -1.408000  
9346  22434600.0  171.565998  172.337000  -0.771002  
9347  23660000.0  172.349997  172.401999  -0.052002  
9348  20225300.0  173.373999  172.336000   1.037999  
9349  21580316.0  173.788001  172.268001   1.520000  


In [6]:
url="https://www.quandl.com/api/v3/datasets/NS1/AAPL_US.csv?api_key=qtu8baDXX4cHaHfrVmSb"
s=requests.get(url).content
df2=pd.read_csv(io.StringIO(s.decode('utf-8')))
df2=df2.convert_objects(convert_numeric=True)
df2["Date"]=pd.to_datetime(df2.Date)
df2=df2.sort_values('Date', ascending=True)
df2=df2.reset_index(drop=True)


c:\users\user\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.


In [7]:
print (df3.tail())

       Adj Close      Volume       label
9335  176.419998  29421100.0  172.229996
9336  174.539993  27436400.0  173.029999
9337  174.350006  23475600.0  175.000000
9338  175.009995  20949900.0  174.350006
9339  175.009995  16349400.0  174.330002


In [8]:
df3.to_csv('forecast.csv')

In [9]:
print (forecast_set)

[ 171.7141752   171.74838666  172.23316448  170.36810757  173.41754523
  173.38599199  174.1935243   176.17564267  175.52269333  175.50209575]


In [10]:
predicted=pd.DataFrame(forecast_set)

In [11]:
predicted.to_csv('predict.csv')